In [1]:
import findspark
findspark.init("/usr/hdp/current/spark2-client/")

In [2]:
from pyspark import SparkContext, SparkConf

In [3]:
spark_conf = SparkConf() \
.setAppName("PairRDDOps_YARN") \
.setMaster("yarn") \
.set('spark.yarn.dist.files','file:/usr/hdp/current/spark2-client/python/lib/pyspark.zip,file:/usr/hdp/current/spark2-client/python/lib/py4j-0.10.4-src.zip') \
.setExecutorEnv('PYTHONPATH','pyspark.zip:py4j-0.10.4-src.zip')

In [4]:
sc = SparkContext(conf=spark_conf)

In [5]:
simple_data_rdd = sc.textFile("/user/erkan/simple_data.csv")

In [6]:
simple_data_rdd.take(3)

['sirano,isim,yas,meslek,sehir,aylik_gelir',
 '1,Cemal,35,Isci,Ankara,3500',
 '2,Ceyda,42,Memur,Kayseri,4200']

In [7]:
# İlk satırdan kurtulma
simple_data_rdd2 = simple_data_rdd.filter(lambda x: 'sirano' not in x)

In [8]:
simple_data_rdd2.take(3)

['1,Cemal,35,Isci,Ankara,3500',
 '2,Ceyda,42,Memur,Kayseri,4200',
 '3,Timur,30,Müzisyen,Istanbul,9000']

# Mesleklere Göre Ortalama Maaşlar

In [9]:
# İhtiyacımız olan meslek ve maaşa ulaşalım
simple_data_rdd2.map(lambda x: (x.split(",")[3], x.split(",")[-1])).take(5)

[('Isci', '3500'),
 ('Memur', '4200'),
 ('Müzisyen', '9000'),
 ('Pazarlamaci', '4200'),
 ('Pazarlamaci', '4800')]

In [12]:
meslek_maas_pair_rdd = simple_data_rdd2.map(lambda x: (x.split(",")[3], float(x.split(",")[-1])))

In [14]:
# Acaba mesleklere göre toplam maaşlar ne kadar?
meslek_maas_pair_rdd.reduceByKey(lambda x,y: x+y).take(15)

[('Memur', 12200.0),
 ('Pazarlamaci', 16300.0),
 ('Tuhafiyeci', 4800.0),
 ('Tornacı', 4200.0),
 ('Isci', 3500.0),
 ('Müzisyen', 29700.0),
 ('Doktor', 32250.0),
 ('Berber', 29600.0)]

In [20]:
# 
meslek_maas_pair_rdd.mapValues(lambda x: (x,1)).take(5)

[('Isci', (3500.0, 1)),
 ('Memur', (4200.0, 1)),
 ('Müzisyen', (9000.0, 1)),
 ('Pazarlamaci', (4200.0, 1)),
 ('Pazarlamaci', (4800.0, 1))]

In [21]:
meslek_maas_pair_rdd.mapValues(lambda x: (x,1)) \
.reduceByKey(lambda x,y: (x[0] + y[0], x[1] + y[1] )) \
.take(5)

[('Memur', (12200.0, 3)),
 ('Pazarlamaci', (16300.0, 3)),
 ('Tuhafiyeci', (4800.0, 1)),
 ('Tornacı', (4200.0, 1)),
 ('Isci', (3500.0, 1))]

In [24]:
meslek_maas_pair_rdd.mapValues(lambda x: (x,1)) \
.reduceByKey(lambda x,y: (x[0] + y[0], x[1] + y[1] )) \
.mapValues(lambda x: (x[0] / x[1])) \
.take(5)

[('Memur', 4066.6666666666665),
 ('Pazarlamaci', 5433.333333333333),
 ('Tuhafiyeci', 4800.0),
 ('Tornacı', 4200.0),
 ('Isci', 3500.0)]

In [30]:
meslek_maas_pair_rdd.mapValues(lambda x: (x,1)) \
.reduceByKey(lambda x,y: (x[0] + y[0], x[1] + y[1] )) \
.mapValues(lambda x: (x[0] / x[1])) \
.map(lambda x: (x[1], x[0])) \
.sortByKey(ascending=False) \
.map(lambda x: (x[1], x[0])) \
.take(5)

[('Doktor', 16125.0),
 ('Müzisyen', 9900.0),
 ('Berber', 9866.666666666666),
 ('Pazarlamaci', 5433.333333333333),
 ('Tuhafiyeci', 4800.0)]